In [61]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [63]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [64]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [65]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [66]:
# Convert String Data to numerical w/ get_dummies
df_binary_encoded = pd.get_dummies(df, drop_first = True)

df_binary_encoded.head(10)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,loan_status_low_risk,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,1,1,0,0,1,1,1,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0,0,0,1,0,1,1,1,1,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0,0,0,1,0,1,1,1,1,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,1,0,1,0,1,1,1,1,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0,0,0,0,0,1,1,1,1,0
5,25000.0,0.1797,634.43,90000.0,18.49,1.0,1.0,14.0,0.0,23889.0,...,0,0,1,0,0,1,1,1,1,0
6,20400.0,0.2000,540.48,51315.0,25.63,0.0,2.0,8.0,1.0,2569.0,...,0,1,1,0,0,1,1,1,1,0
7,10000.0,0.1557,349.46,66000.0,9.00,0.0,0.0,11.0,0.0,9407.0,...,1,0,1,0,0,1,1,1,1,0
8,14000.0,0.0881,443.96,45000.0,10.24,0.0,1.0,9.0,0.0,7058.0,...,0,1,0,0,0,1,1,0,1,0
9,10000.0,0.1474,345.39,63295.0,39.34,0.0,0.0,27.0,0.0,25790.0,...,0,1,0,1,0,1,1,1,1,0


In [67]:
# Get outcomes
df_binary_encoded.rename(columns={"loan_status_low_risk":"loan_status"}, inplace=True)

df_binary_encoded.head()

y = df_binary_encoded["loan_status"].ravel()


# Get features
X = df_binary_encoded.drop(columns = "loan_status")

In [69]:
df_binary_encoded.head()


,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,loan_status,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,1,1,0,0,1,1,1,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0,0,0,1,0,1,1,1,1,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0,0,0,1,0,1,1,1,1,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,1,0,1,0,1,1,1,1,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0,0,0,0,0,1,1,1,1,0


# Split the Data into Training and Testing

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [71]:
from sklearn.preprocessing import StandardScaler
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [72]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
y_pred = rf_model.predict(X_test_scaled)

In [73]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.6570243445740461

In [79]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,29,63
Actual 1,20,17093


In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.59      0.32      1.00      0.41      0.56      0.29        92
          1       1.00      1.00      0.32      1.00      0.56      0.34     17113

avg / total       0.99      1.00      0.32      0.99      0.56      0.34     17205



In [32]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
importances
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08663294781210748, 'total_pymnt_inv'),
 (0.0814462605713496, 'total_rec_int'),
 (0.07325380199586823, 'total_pymnt'),
 (0.0691728557206635, 'last_pymnt_amnt'),
 (0.06419515532095707, 'total_rec_prncp'),
 (0.018720337466275733, 'mo_sin_old_rev_tl_op'),
 (0.018035260193828954, 'out_prncp_inv'),
 (0.017459613777580908, 'installment'),
 (0.0164944490764097, 'total_rec_late_fee'),
 (0.0162210219879808, 'out_prncp'),
 (0.015942024573197763, 'max_bal_bc'),
 (0.015581830290552845, 'dti'),
 (0.015156937877593216, 'loan_amnt'),
 (0.01458983716874935, 'mo_sin_old_il_acct'),
 (0.014258443372915899, 'annual_inc'),
 (0.014209084764614471, 'total_bc_limit'),
 (0.014095138455012873, 'avg_cur_bal'),
 (0.014063058172602602, 'bc_util'),
 (0.014050940484714496, 'bc_open_to_buy'),
 (0.013566237499976225, 'revol_bal'),
 (0.01281424789337213, 'tot_hi_cred_lim'),
 (0.012460086765927511, 'total_rev_hi_lim'),
 (0.011973424338117947, 'total_bal_ex_mort'),
 (0.01192207950169501, 'total_bal_il'),
 (0.011633390

### Easy Ensemble AdaBoost Classifier

In [85]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

rf_model = EasyEnsembleClassifier(n_estimators=100, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
y_pred = rf_model.predict(X_test_scaled)

In [86]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9226293765990259

In [87]:
# Display the confusion matrix
y_pred = rf_model.predict(X_test)
confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
#cm_df = pd.DataFrame(
#    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

#cm_df

array([[   78,    14],
       [14127,  2986]], dtype=int64)

In [88]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.85      0.17      0.01      0.38      0.16        92
          1       1.00      0.17      0.85      0.30      0.38      0.14     17113

avg / total       0.99      0.18      0.84      0.30      0.38      0.14     17205

